In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import json
from datetime import date

# Функция для чтения выборки из большого файла (например, train.csv) по частям.
# Мы читаем sample_size строк, используя чанки по chunksize строк – это позволяет сократить время обработки.
def read_csv_sample(file_path, sample_size=1000000, chunksize=100000):
    chunks = []
    for chunk in tqdm(pd.read_csv(file_path, chunksize=chunksize),
                      total=sample_size//chunksize,
                      desc="Чтение файла"):
        chunks.append(chunk)
        if len(chunks) * chunksize >= sample_size:
            break
    return pd.concat(chunks, ignore_index=True).head(sample_size)

# Функция для рекурсивного преобразования ключей типа date в строки внутри словаря
def convert_dates_to_strings(d):
    if isinstance(d, dict):
        return {(str(k) if isinstance(k, date) else k): convert_dates_to_strings(v) for k, v in d.items()}
    elif isinstance(d, list):
        return [convert_dates_to_strings(v) for v in d]
    elif isinstance(d, date):
        return d.isoformat()
    else:
        return d

# Загрузка данных. Для train.csv используется выборка для сокращения времени работы.
print("Загрузка данных...")
train_df = read_csv_sample('train.csv')  # выборка из train.csv (примерно 1 млн строк)
questions_df = pd.read_csv('questions.csv')
lectures_df = pd.read_csv('lectures.csv')

# Инициализация словаря для хранения результатов анализа
analysis_results = {}

# ================================
# 1. Анализ train.csv
# ================================
print("Анализ train.csv...")
analysis_results['train_stats'] = train_df.describe().to_dict()
analysis_results['correlations'] = train_df.corr().to_dict()

# Преобразование timestamp в datetime для анализа временных трендов
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], unit='ms')
train_df['date'] = train_df['timestamp'].dt.date

# Средняя успеваемость по дням – позволяет изучить динамику результатов.
daily_performance = train_df.groupby('date')['answered_correctly'].mean()
analysis_results['daily_performance'] = {str(k): v for k, v in daily_performance.items()}  # преобразуем даты в строки

# Анализ влияния наличия объяснения на успеваемость
explanation_impact = train_df.groupby('prior_question_had_explanation')['answered_correctly'].mean()
analysis_results['explanation_impact'] = explanation_impact.to_dict()

# ================================
# 2. Анализ questions.csv
# ================================
print("Анализ questions.csv...")
part_distribution = questions_df['part'].value_counts()
analysis_results['part_distribution'] = part_distribution.to_dict()

# Разбиваем столбец с тегами, затем объединяем для определения топ-20 тегов
tags = questions_df['tags'].str.split().explode()
top_tags = tags.value_counts().head(20)
analysis_results['top_tags'] = top_tags.to_dict()

# ================================
# 3. Анализ lectures.csv
# ================================
print("Анализ lectures.csv...")
lecture_types = lectures_df['type_of'].value_counts()
analysis_results['lecture_types'] = lecture_types.to_dict()

# ================================
# 4. Объединение данных и дополнительный анализ
# ================================
print("Объединение данных...")
merged_df = pd.merge(train_df, questions_df, left_on='content_id', right_on='question_id', how='left')

# Анализ успеваемости по частям теста на основе объединённых данных
part_performance = merged_df.groupby('part')['answered_correctly'].mean()
analysis_results['part_performance'] = part_performance.to_dict()

# ================================
# 5. Дополнительный анализ
# ================================
# Анализ успеваемости в зависимости от типа контента: 0 - вопрос, 1 - лекция.
content_type_performance = train_df.groupby('content_type_id')['answered_correctly'].mean()
analysis_results['content_type_performance'] = content_type_performance.to_dict()

# Анализ влияния количества попыток пользователя (attempts) на успеваемость
attempts_performance = train_df.groupby('user_id').agg({
    'content_id': 'count',
    'answered_correctly': 'mean'
}).rename(columns={'content_id': 'attempts'})
analysis_results['attempts_correlation'] = attempts_performance['attempts'].corr(attempts_performance['answered_correctly'])

# Анализ успеваемости по времени суток – помогает выявить оптимальные часы для занятий.
train_df['hour'] = train_df['timestamp'].dt.hour
hourly_performance = train_df.groupby('hour')['answered_correctly'].mean()
analysis_results['hourly_performance'] = hourly_performance.to_dict()

# Анализ успеваемости по тегам
tag_performance = merged_df.groupby('tags')['answered_correctly'].mean().sort_values(ascending=False)
analysis_results['top_10_tags_performance'] = tag_performance.head(10).to_dict()

# Преобразуем даты в строки во всех структурах данных (если встречаются объекты типа date)
analysis_results = convert_dates_to_strings(analysis_results)

# Сохраняем результаты анализа в JSON-файл для удобной передачи и хранения.
with open('analysis_results.json', 'w') as f:
    json.dump(analysis_results, f)

print("Анализ завершен. Результаты сохранены в файл 'analysis_results.json'.")

# ================================
# 6. Финальный вывод результатов и построение отчёта
# ================================
print("\n--- Финальные результаты анализа ---\n")

# Загружаем результаты из файла
with open('analysis_results.json', 'r') as f:
    results = json.load(f)

# Создаем список строк для финального отчета
final_report_lines = []

# Общая статистика по train.csv
final_report_lines.append("=== Общая статистика по train.csv ===")
for col, stats in results.get("train_stats", {}).items():
    final_report_lines.append(f"Колонка '{col}':")
    for stat_name, stat_value in stats.items():
        final_report_lines.append(f"  {stat_name}: {stat_value}")
    final_report_lines.append("")

# Корреляции между признаками
final_report_lines.append("=== Корреляции между признаками train.csv ===")
corr_dict = results.get("correlations", {})
high_corr_pairs = []
processed_pairs = set()
for k, subdict in corr_dict.items():
    for k2, corr_val in subdict.items():
        if k != k2 and abs(corr_val) >= 0.7 and (k2, k) not in processed_pairs:
            high_corr_pairs.append((k, k2, corr_val))
            processed_pairs.add((k, k2))
if high_corr_pairs:
    final_report_lines.append("Пары признаков с высокой корреляцией (>0.7):")
    for pair in high_corr_pairs:
        final_report_lines.append(f"  {pair[0]} и {pair[1]}: {pair[2]:.3f}")
else:
    final_report_lines.append("Нет пар признаков с корреляцией выше 0.7.")
final_report_lines.append("")

# Ежедневная успеваемость
final_report_lines.append("=== Средняя успеваемость по дням ===")
for d, perf in sorted(results.get("daily_performance", {}).items()):
    final_report_lines.append(f"{d}: {perf:.3f}")
final_report_lines.append("")

# Влияние объяснения
final_report_lines.append("=== Влияние наличия объяснения на успеваемость ===")
for key, val in results.get("explanation_impact", {}).items():
    final_report_lines.append(f"{key}: {val:.3f}")
final_report_lines.append("")

# Распределение вопросов по частям теста
final_report_lines.append("=== Распределение вопросов по частям теста ===")
for part, count in results.get("part_distribution", {}).items():
    final_report_lines.append(f"Часть {part}: {count} вопросов")
final_report_lines.append("")

# Топ-20 тегов вопросов
final_report_lines.append("=== Топ-20 тегов вопросов ===")
for tag, count in results.get("top_tags", {}).items():
    final_report_lines.append(f"Тег {tag}: {count}")
final_report_lines.append("")

# Распределение лекций по типам
final_report_lines.append("=== Распределение лекций по типам ===")
for lec_type, count in results.get("lecture_types", {}).items():
    final_report_lines.append(f"{lec_type}: {count}")
final_report_lines.append("")

# Успеваемость по частям теста (объединённые данные)
final_report_lines.append("=== Средняя успеваемость по частям теста (merged data) ===")
for part, perf in results.get("part_performance", {}).items():
    final_report_lines.append(f"Часть {part}: {perf:.3f}")
final_report_lines.append("")

# Успеваемость по типу контента
final_report_lines.append("=== Успеваемость по типу контента (0 – вопрос, 1 – лекция) ===")
for ct, perf in results.get("content_type_performance", {}).items():
    final_report_lines.append(f"Тип {ct}: {perf:.3f}")
final_report_lines.append("")

# Корреляция между количеством попыток и успеваемостью
final_report_lines.append("=== Корреляция между количеством попыток и успеваемостью ===")
final_report_lines.append(f"Коэффициент корреляции: {results.get('attempts_correlation', 0):.3f}")
final_report_lines.append("")

# Успеваемость по часам суток
final_report_lines.append("=== Средняя успеваемость по часам (0-23) ===")
for hour, perf in sorted(results.get("hourly_performance", {}).items(), key=lambda x: int(x[0])):
    final_report_lines.append(f"Час {hour}: {perf:.3f}")
final_report_lines.append("")

# Топ-10 тегов по успеваемости
final_report_lines.append("=== Топ-10 тегов по успеваемости ===")
for tag, perf in results.get("top_10_tags_performance", {}).items():
    final_report_lines.append(f"Тег {tag}: {perf:.3f}")
final_report_lines.append("")

# Генерация итоговых выводов и рекомендаций
final_report_lines.append("=== Выводы и рекомендации ===")
conclusions = [
    "Анализ типа контента показывает, что вопросы (content_type=0) имеют более высокую успеваемость, чем лекции (content_type=1).",
    "Количество попыток демонстрирует умеренную корреляцию с успеваемостью, что свидетельствует о влиянии активности пользователей на результаты.",
    "Успеваемость студентов по часам существенно варьируется – наилучшие результаты наблюдаются в определенные часы, что говорит о важности оптимального времени для занятий.",
    "Анализ тегов выявил, что среди топ-10 тегов присутствуют те, которые ассоциируются с высокими результатами; их можно использовать для формирования рекомендаций по дополнительным материалам.",
    "Части теста с большим количеством вопросов, например части 1 и 5, требуют особого внимания при разработке системы поддержки обучения."
]
for idx, item in enumerate(conclusions, 1):
    final_report_lines.append(f"{idx}. {item}")
final_report_lines.append("")

final_report_lines.append("Рекомендации:")
final_report_lines.append(" - Адаптировать тип контента под индивидуальные предпочтения пользователей.")
final_report_lines.append(" - Учитывать оптимальное время для проведения занятий для повышения успеваемости.")
final_report_lines.append(" - Использовать анализ тегов для выдачи персонализированных рекомендаций по материалам.")
final_report_lines.append(" - Дополнительно изучать влияние количества попыток на успех пользователей.")
final_report_lines.append("")

# Сохранение итогового отчета в отдельный текстовый файл
with open("final_conclusions.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(final_report_lines))

# Вывод итогового отчета в консоль для удобства чтения
print("\n".join(final_report_lines))


Загрузка данных...


Чтение файла:  90%|█████████ | 9/10 [00:01<00:00,  8.07it/s]


Анализ train.csv...
Анализ questions.csv...
Анализ lectures.csv...
Объединение данных...
Анализ завершен. Результаты сохранены в файл 'analysis_results.json'.

--- Финальные результаты анализа ---

=== Общая статистика по train.csv ===
Колонка 'row_id':
  count: 1000000.0
  mean: 499999.5
  std: 288675.27893349814
  min: 0.0
  25%: 249999.75
  50%: 499999.5
  75%: 749999.25
  max: 999999.0

Колонка 'timestamp':
  count: 1000000.0
  mean: 7344311266.729203
  std: 10586930270.349688
  min: 0.0
  25%: 521166935.5
  50%: 2824811074.0
  75%: 10101863021.25
  max: 78091996556.0

Колонка 'user_id':
  count: 1000000.0
  mean: 10172549.789739
  std: 6029233.8638747
  min: 115.0
  25%: 4702585.0
  50%: 9678259.0
  75%: 15568723.0
  max: 20949024.0

Колонка 'content_id':
  count: 1000000.0
  mean: 5232.691523
  std: 3882.323419315062
  min: 0.0
  25%: 2057.0
  50%: 5052.0
  75%: 7421.0
  max: 32736.0

Колонка 'content_type_id':
  count: 1000000.0
  mean: 0.019907
  std: 0.13968081780178743
  min: